In [1]:
import torch.nn as nn
import torch
import numpy as np
import torchvision
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
from NeuralNet import NeuralNetwork
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [2]:
learning_rate = .1
batch_size = 32
epochs = 40

In [3]:
train_data = MNIST('data',train=True,download=True, transform=torchvision.transforms.ToTensor())
test_data = MNIST('data',train=False,download=True, transform=torchvision.transforms.ToTensor())
mnist_train = DataLoader(train_data, batch_size=batch_size, shuffle=True)
mnist_test = DataLoader(test_data, batch_size=batch_size,shuffle=True)

In [4]:
model = NeuralNetwork()
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer)

In [5]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    avg_loss = np.empty(0)
    for batch, (X, y) in enumerate(dataloader):
        pred = model(X)
        loss = loss_fn(pred, y)
        avg_loss = np.append(avg_loss, loss.item())
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
    scheduler.step(torch.mean(torch.tensor(avg_loss)))

In [6]:
def test_loop(dataloader, model, loss_fn):
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [7]:
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(mnist_train, model, loss, optimizer)
    test_loop(mnist_test, model, loss)
print("Done!")

Epoch 1
-------------------------------
loss: 2.305026  [   32/60000]
loss: 2.277832  [ 3232/60000]
loss: 2.053226  [ 6432/60000]
loss: 1.121153  [ 9632/60000]
loss: 0.727683  [12832/60000]
loss: 0.637947  [16032/60000]
loss: 0.474825  [19232/60000]
loss: 0.531527  [22432/60000]
loss: 0.878165  [25632/60000]
loss: 0.244950  [28832/60000]
loss: 0.314103  [32032/60000]
loss: 0.214694  [35232/60000]
loss: 0.236980  [38432/60000]
loss: 0.367572  [41632/60000]
loss: 0.359523  [44832/60000]
loss: 0.352870  [48032/60000]
loss: 0.835158  [51232/60000]
loss: 0.296905  [54432/60000]
loss: 0.379312  [57632/60000]
Test Error: 
 Accuracy: 89.2%, Avg loss: 0.344620 

Epoch 2
-------------------------------
loss: 0.134580  [   32/60000]
loss: 0.241414  [ 3232/60000]
loss: 0.210539  [ 6432/60000]
loss: 0.299511  [ 9632/60000]
loss: 0.122107  [12832/60000]
loss: 0.225581  [16032/60000]
loss: 0.237514  [19232/60000]
loss: 0.257810  [22432/60000]
loss: 0.175582  [25632/60000]
loss: 0.097909  [28832/60000

In [9]:
torch.save(model,'models/10w1a32a40.pt')